In [1]:
from itk import *
from vtk import *

In [9]:
# im = imread("./data/BRATS_HG0015_T1C.mha")
# im = GetArrayFromImage(im)
# im.max()

2592

In [112]:
def _check_valid_arg(val, name, available):
    if val not in available:
        raise f"{name}='{val}' is not a valid arg. Valid values are: {available}"
        
def load_volume(reader, min_intensity=0, intensity_coeff=0.5, color=(1.,1.,1.), opacity_window=4096, render_with="gl", interpolation="linear"):
    _check_valid_arg(render_with, "render_with", {'gl', 'gpu', 'cpu'})
    _check_valid_arg(interpolation, "interpolation", {'linear', 'nearest'})
    
    reader.Update()

    if render_with == "gl":
        mapper = vtkOpenGLGPUVolumeRayCastMapper()
    elif render_with == 'gpu':
        mapper = vtkGPUVolumeRayCastMapper() 
    elif render_with == 'cpu':
        mapper = vtkFixedPointVolumeRayCastMapper()
    else:
        raise "unexpected"
        
    mapper.SetInputConnection(reader.GetOutputPort())
    mapper.SetSampleDistance(0.5)
    mapper.SetBlendModeToMaximumIntensity()

    data_min, data_max = reader.GetOutput().GetScalarRange()
    
    seg_min, seg_max = min_intensity, min_intensity + (1-intensity_coeff) * (data_max-min_intensity)
    fct_color = vtkColorTransferFunction()
    fct_color.AddRGBSegment(seg_min, *(0., 0., 0.),
                            seg_max, *color)
    
    fct_opacity = vtkPiecewiseFunction()
    fct_opacity.AddSegment(seg_min, 0.,
                           seg_max, 1.)
    
    props = vtkVolumeProperty()
    props.SetIndependentComponents(True)
    props.SetColor(fct_color)
    props.SetScalarOpacity(fct_opacity)

    if interpolation == "linear":
        props.SetInterpolationTypeToLinear()
    elif interpolation == 'nearest':
        props.SetInterpolationTypeToNearest()
    else:
        raise "unexpected"

    volume = vtkVolume()
    volume.SetMapper(mapper)
    volume.SetProperty(props)
    
    return volume, props

In [111]:
reader = vtkMetaImageReader()
reader.SetFileName("./data/BRATS_HG0015_T1C.mha")

1.46 µs ± 32.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [115]:
volume, _ = load_volume(reader, intensity_coeff=0.4)
volume_red, _ = load_volume(reader, intensity_coeff=0.8, min_intensity=1000, color=[1,0,0])

ren = vtkRenderer()
ren.AddVolume(volume)
ren.AddVolume(volume_red)

renWin = vtkRenderWindow()
renWin.AddRenderer(ren)

iren = vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
iren.Initialize()

# iren.AddObserver('TimerEvent', cb.execute)
# timerId = iren.CreateRepeatingTimer(100);

renWin.Render()

iren.Start()